<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/02_prompt_engineering.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Prompt Engineering

## Overview  
In this exercise, we will explore prompt engineering within the Solar framework. Prompt engineering is a crucial technique in leveraging large language models effectively by crafting prompts that elicit the desired responses from the model. This tutorial will walk you through various strategies for creating and refining prompts to optimize the performance of Solar in different NLP tasks.
 
## Purpose of the Exercise
The purpose of this exercise is to equip users with practical skills in prompt engineering. By the end of this tutorial, users will be able to design effective prompts, understand the impact of prompt variations, and enhance the accuracy and relevance of responses generated by the Solar LLM. This foundational knowledge is essential for advanced applications and fine-tuning of language models.


In [4]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython
 
upstage_api_key_env_name = "up_svEvm8FOA6g9OILcC8oqs9bg4RSB7"


def load_env():
    if "google.colab" in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata

        upstage_api_key = userdata.get(upstage_api_key_env_name)
        return os.environ.setdefault(upstage_api_key_env_name, upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv

        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)


UPSTAGE_API_KEY = load_env()

https://api.python.langchain.com/en/latest/chat_models/langchain_upstage.chat_models.ChatUpstage.html

미니 모델이 맞대요 .

In [5]:
# Quick hello world 
from langchain_upstage import ChatUpstage

llm = ChatUpstage(api_key=upstage_api_key_env_name)
response = llm.invoke("What is the capital of France")

print(response) # 답변 외에도 token 수,, 거절 했는지 여부 등 여러 정보가 출력됨
print(response.content) # 얘가 generate 한 content만 뽑아내는 것

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 16, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-mini-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8c66c301-9df0-498f-81f9-3c0fa128b5ac-0' usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}}
The capital of France is Paris.


In [6]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."), # defult prompt # 여기다가 형식 요구해도 되고
        ("human", "What is the capital of France?"),
        ("ai", "!Answer! It's Paris!!"), # chat history를 넣어주는 것,, in-context learning # 여기다가 form 넣어줘도 되고
        ("human", "What about Korea?"), #여따 해도 되고 ~ 여러가지 잇음~
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm 
response = chain.invoke({})

print(response.content)

!Answer! The capital of South Korea is Seoul. The capital of North Korea is Pyongyang. Remember, these are two different countries with different capitals!


In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template( # basic prompt
    """
Q: The cafeteria had 23 apples. 
If they used 20 to make lunch and bought 6 more, 
how many apples do they have?

A: the answer is 
"""
    # zero-shot learning
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

The cafeteria started with 23 apples. They used 20 of them, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they had 3 + 6 = 9 apples. 

The final answer is: \boxed{9}


In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
) # one-shot prompting
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

19.

The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they had 3 + 6 = 9 apples. However, since they used 20 apples for lunch, they would have 9 - 20 = -11 apples, which is not possible. Therefore, the cafeteria would have run out of apples before they could make lunch for everyone.


![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

The cafeteria started with 23 apples. They used 20, so they had 3 apples left. They bought 6 more, so they had 9 apples. The answer is 9.


![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

Step 1: First, we need to determine the number of apples used for lunch. The cafeteria used 20 apples.
Step 2: Next, we calculate the number of apples remaining after using 20. The cafeteria started with 23 apples, so after using 20, they would have 23 - 20 = 3 apples left.
Step 3: Finally, we add the 6 apples they bought to the 3 apples remaining. This gives us a total of 3 + 6 = 9 apples.

Answer: The cafeteria has 9 apples.


## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [60]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)


1. What is SOLAR 10.7B and what are its key features?
2. How does depth up-scaling (DUS) differ from other LLM up-scaling methods, and what are its advantages?
3. How does SOLAR 10.7B-Instruct compare to Mixtral-8x7B-Instruct in terms of instruction-following capabilities?


In [61]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

1. SOLAR 10.7B
2. depth up-scaling (DUS)
3. instruction-following capabilities


In [62]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

What is the method for scaling large language models (LLMs) called, which encompasses depthwise scaling and continued pretraining, and does not require complex changes to train and inference efficiently?


In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [13]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

'Why did the chicken join a band?\n\nBecause it had the drumsticks!'

In [14]:
chain.invoke({"adjective": "funny", "content": "beef"})

'Why did the beef go to the gym? To beef up!'

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the name of the large language model with 10.7 billion parameters introduced in the text?'

# References
* https://platform.openai.com/docs/guides/prompt-engineering 
* https://docs.anthropic.com/claude/docs/intro-to-prompting 
* https://smith.langchain.com/hub 